# Simplified fly model

In [1]:
%reset -f
%load_ext autoreload
%autoreload 2

import sys
import os

sys.path.append(os.path.join(os.path.pardir, 'code'))

import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

from symbolic_derivatives import directional_derivative, chain_rule_derivative, get_bigO, get_vars

In [2]:
sin = sp.sin
cos = sp.cos
tan = sp.tan

In [3]:
sp.init_printing()

### Define variables

In [4]:
W, V, D, phi, zeta = sp.symbols(['W', 'V', 'D', 'phi', 'zeta'])

u_V, u_phi = sp.symbols(['u_V', 'u_phi'])

k, = sp.symbols(['k',])

### Airspeed

In [5]:
ax = -V*cos(phi) + W*cos(zeta)
ay = -V*sin(phi) + W*sin(zeta)

### Observability

In [6]:
x = [D, V, W, phi, zeta]
x

In [7]:
f_0 = [0, 0, 0, 0, 0]
sp.Matrix(f_0)

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

In [8]:
f_V = [0, 1, 0, 0, 0]
sp.Matrix(f_V)

⎡0⎤
⎢ ⎥
⎢1⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

In [9]:
f_phi = [0, 0, 0, 1, 0]
sp.Matrix(f_phi)

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢1⎥
⎢ ⎥
⎣0⎦

In [10]:
f_0 = sp.Matrix(f_0)
f_V = sp.Matrix(f_V)
f_phi = sp.Matrix(f_phi)

In [11]:
full_dynamics = f_0 + u_phi*f_phi + u_V*f_V
full_dynamics

⎡ 0 ⎤
⎢   ⎥
⎢u_V⎥
⎢   ⎥
⎢ 0 ⎥
⎢   ⎥
⎢uᵩ ⎥
⎢   ⎥
⎣ 0 ⎦

### Some values

In [12]:
s = {V: 11, W: 7, D: 5, phi: 3, zeta: 2, k: 23}

us = {u_V: 13, u_phi: 17}

### Observability analysis

In [13]:
h = sp.Matrix([ax/ay, 
               phi, 
               V/D
              ])
h

⎡-V⋅cos(φ) + W⋅cos(ζ)⎤
⎢────────────────────⎥
⎢-V⋅sin(φ) + W⋅sin(ζ)⎥
⎢                    ⎥
⎢         φ          ⎥
⎢                    ⎥
⎢         V          ⎥
⎢         ─          ⎥
⎣         D          ⎦

### With just h

In [14]:
O = h
jx = get_vars(sp.Matrix.vstack(O, sp.Matrix([x]).T ) )
J = O.jacobian( jx ).subs(s)
R_nozeta = np.linalg.matrix_rank(np.matrix(J, dtype=float))

# augment with zeta
Ozeta = O.row_insert(0, sp.Matrix([zeta]))
Jzeta = Ozeta.jacobian( jx ).subs(s)
R_zeta = np.linalg.matrix_rank(np.matrix(Jzeta, dtype=float))

print('Rank O = ', R_nozeta)
print('Rank {O; zeta} = ', R_zeta)
print()

if R_nozeta == R_zeta:
    print('Observable!')
else:
    print('Not observable.')

    print()
print('x_tilde = ')
jx

Rank O =  3
Rank {O; zeta} =  4

Not observable.

x_tilde = 


### With one derivative, no control

In [15]:
O = h
bigO = get_bigO(h, 
                x, 
                [f_0], 
               )
O = sp.Matrix.vstack(*bigO) 
jx = get_vars(sp.Matrix.vstack(O, sp.Matrix([x]).T ) )
J = O.jacobian( jx ).subs(s)
R_nozeta = np.linalg.matrix_rank(np.matrix(J, dtype=float))

# augment with zeta
Ozeta = O.row_insert(0, sp.Matrix([zeta]))
Jzeta = Ozeta.jacobian( jx ).subs(s)
R_zeta = np.linalg.matrix_rank(np.matrix(Jzeta, dtype=float))

print('Rank O = ', R_nozeta)
print('Rank {O; zeta} = ', R_zeta)
print()

if R_nozeta == R_zeta:
    print('Observable!')
else:
    print('Not observable.')

    print()
print('x_tilde = ')
jx

Rank O =  3
Rank {O; zeta} =  4

Not observable.

x_tilde = 


In [16]:
sp.simplify(O)

⎡V⋅cos(φ) - W⋅cos(ζ)⎤
⎢───────────────────⎥
⎢V⋅sin(φ) - W⋅sin(ζ)⎥
⎢                   ⎥
⎢         φ         ⎥
⎢                   ⎥
⎢         V         ⎥
⎢         ─         ⎥
⎢         D         ⎥
⎢                   ⎥
⎢         0         ⎥
⎢                   ⎥
⎢         0         ⎥
⎢                   ⎥
⎣         0         ⎦

### With one derivative, active u_V

In [17]:
O = h
bigO = get_bigO(h, 
                x, 
                [f_0, f_V], 
               )
O = sp.Matrix.vstack(*bigO) 
jx = get_vars(sp.Matrix.vstack(O, sp.Matrix([x]).T ) )
J = O.jacobian( jx ).subs(s)
R_nozeta = np.linalg.matrix_rank(np.matrix(J, dtype=float))

# augment with zeta
Ozeta = O.row_insert(0, sp.Matrix([zeta]))
Jzeta = Ozeta.jacobian( jx ).subs(s)
R_zeta = np.linalg.matrix_rank(np.matrix(Jzeta, dtype=float))

print('Rank O = ', R_nozeta)
print('Rank {O; zeta} = ', R_zeta)
print()

if R_nozeta == R_zeta:
    print('zeta Observable!')
else:
    print('zeta Not observable.')

    print()
print('x_tilde = ')
jx

Rank O =  5
Rank {O; zeta} =  5

zeta Observable!
x_tilde = 


### with one derivative, active u_phi

In [18]:
O = h
bigO = get_bigO(h, 
                x, 
                [f_0, f_phi], 
               )
O = sp.Matrix.vstack(*bigO) 
jx = get_vars(sp.Matrix.vstack(O, sp.Matrix([x]).T ) )
J = O.jacobian( jx ).subs(s)
R_nozeta = np.linalg.matrix_rank(np.matrix(J, dtype=float))

# augment with zeta
Ozeta = O.row_insert(0, sp.Matrix([zeta]))
Jzeta = Ozeta.jacobian( jx ).subs(s)
R_zeta = np.linalg.matrix_rank(np.matrix(Jzeta, dtype=float))

print('Rank O = ', R_nozeta)
print('Rank {O; zeta} = ', R_zeta)
print()

if R_nozeta == R_zeta:
    print('zeta Observable!')
else:
    print('zeta Not observable.')

    print()
print('x_tilde = ')
jx

Rank O =  4
Rank {O; zeta} =  4

zeta Observable!
x_tilde = 


### with two derivatives, with f_phi active

In [19]:
O = h
bigO = get_bigO(h, 
                x, 
                [f_0, f_phi], 
               )

bigO2 = get_bigO(sp.Matrix.vstack(*bigO), x, [f_0])
O = sp.Matrix.vstack(*bigO, *bigO2)

jx = get_vars(sp.Matrix.vstack(O, sp.Matrix([x]).T ) )
J = O.jacobian( jx ).subs(s)
R_nozeta = np.linalg.matrix_rank(np.matrix(J, dtype=float))

# augment with zeta
Ozeta = O.row_insert(0, sp.Matrix([zeta]))
Jzeta = Ozeta.jacobian( jx ).subs(s)
R_zeta = np.linalg.matrix_rank(np.matrix(Jzeta, dtype=float))

print('Rank O = ', R_nozeta)
print('Rank {O; zeta} = ', R_zeta)
print()

if R_nozeta == R_zeta:
    print('zeta Observable!')
else:
    print('zeta Not observable.')

    print()
print('x_tilde = ')
jx

Rank O =  4
Rank {O; zeta} =  4

zeta Observable!
x_tilde = 


### with two derivatives, f_V active

In [20]:
O = h
bigO = get_bigO(h, 
                x, 
                [f_0, f_V], 
               )

bigO2 = get_bigO(sp.Matrix.vstack(*bigO), x, [f_0])
O = sp.Matrix.vstack(*bigO, *bigO2)

jx = get_vars(sp.Matrix.vstack(O, sp.Matrix([x]).T ) )
J = O.jacobian( jx ).subs(s)
R_nozeta = np.linalg.matrix_rank(np.matrix(J, dtype=float))

# augment with zeta
Ozeta = O.row_insert(0, sp.Matrix([zeta]))
Jzeta = Ozeta.jacobian( jx ).subs(s)
R_zeta = np.linalg.matrix_rank(np.matrix(Jzeta, dtype=float))

print('Rank O = ', R_nozeta)
print('Rank {O; zeta} = ', R_zeta)
print()

if R_nozeta == R_zeta:
    print('zeta Observable!')
else:
    print('zeta Not observable.')

    print()
print('x_tilde = ')
jx

Rank O =  5
Rank {O; zeta} =  5

zeta Observable!
x_tilde = 
